# Generate LOpInf roms

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from sklearn.utils.extmath import randomized_svd
 
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.animation import FuncAnimation
from dotenv import dotenv_values 
from moviepy.editor import VideoClip
from moviepy.video.io.bindings import mplfig_to_npimage
import scipy.io
from scipy.signal import cont2discrete
import pyvista as pv
import h5py
import mat73
import dask.array as da
from dask.distributed import Client, LocalCluster
import opinf
config = dotenv_values(".env")
from util import *

In [2]:
####### Set up dask cluster #######
# Setup Dask client
# cluster = LocalCluster() # Launches a scheduler and workers locally
# client = Client(cluster) # Connect to distributed cluster and override default
client = Client('tcp://127.0.0.1:8786')
client.cluster

####### # Set up hyperparameters for all ROMs  #######
n_timesteps = 2000 # Number of timesteps to pull from each episode
dt = 0.001
n_train = 3 # Number of training episodes
ns_ROM = np.arange(2,202,2) # Dimensions of ROM state space

In [3]:
####### Set up filepaths #######
# Setup filepaths for reading data 
filepath = config["currentDirectory"] + "data/archivedDataSets/FullAssembly_Constrained_FullSetForRAL_goodMatParams/"
romDir = config["currentDirectory"] + "data/archivedDataSets/FullAssembly_Constrained_FullSetForRAL_goodMatParams/ROMs/"

####### Load in data #######
print("Loading in data...")
# Read in all training data 
data = h5py.File(filepath+ "fullDataSet.hdf5", 'r')
X_fom = da.from_array(data["stateData"], chunks=(4096, 4096,1))
X_fom = X_fom[:,:,0:n_train]
U_fom = da.from_array(data["inputData"], chunks=(4096, 4096,1))
U_fom = U_fom[:,:,0:n_train]
Y_fom = da.from_array(data["reducedCenterlineData"], chunks=(4096, 4096,1))
Y_fom = Y_fom[:,:,0:n_train]
x0 = data["stateData"][:,0,0]
y0 = data["reducedCenterlineData"][:,0,0]
n = X_fom.shape[0]
l = U_fom.shape[0]
m = Y_fom.shape[0]
# # Collect initial condition offsets for each episode and center each episode
X0 = da.zeros((n,n_train))
Y0 = da.zeros((m,n_train))
for i in range(n_train):
    X0[:,i] = X_fom[:,0,i]
    Y0[:,i] = Y_fom[:,0,i]
    X_fom[:,:,i] = X_fom[:,:,i] - X0[:,i].reshape(-1,1)
    Y_fom[:,:,i] = Y_fom[:,:,i] - Y0[:,i].reshape(-1,1)
print("Done loading in data.")

Loading in data...
Done loading in data.


In [4]:
# Read in .mat file for LOpInf ROM
lOpInfMats = scipy.io.loadmat(romDir + "lopinf_rom_operators_shifted.mat")
lOpInfbases = scipy.io.loadmat(romDir+'lopinf_shifted_basis_nkt_3.mat')
        


In [5]:
nROMs_LOpInf = np.array([2,4,6,8,10])

In [6]:
U_fom = U_fom.compute()
for i in range(len(nROMs_LOpInf)):
    n_ROM = nROMs_LOpInf[i]*2
    # Grab matrices from .mat file
    Bhat_lopinf = lOpInfMats['ROM_B'][0,i]
    Chat_lopinf = lOpInfMats['ROM_C'][0,i]
    Khat_lopinf = lOpInfMats['ROM_K'][0,i]
    Mhat_lopinf = lOpInfMats['ROM_M'][0,i]
    basis_lopinf = np.hstack((lOpInfbases['V'][:,:n_ROM//2],np.zeros((lOpInfbases['V'][:,:n_ROM//2].shape[0],n_ROM//2))))

    # Form continuous time linear state transition matrices for LOpInf ROM
    A_lopinf_ct = np.block([
        [np.zeros((n_ROM//2,n_ROM//2)),np.eye(n_ROM//2)],
        [-Khat_lopinf, -Chat_lopinf]
    ])
    B_lopinf_ct = np.block([
        [np.zeros((n_ROM//2,l))],
        [Bhat_lopinf]
    ])
    # Convert continuous time operators to discrete time operators
    A_lopinf_dt = scipy.linalg.expm(A_lopinf_ct*dt)
    B_lopinf_dt = (A_lopinf_dt-np.eye(n_ROM))@np.linalg.inv(A_lopinf_ct)@B_lopinf_ct

    # Initialize matrices to hold states so that we can compute the output matrices
    X_lopinf = np.zeros((n_ROM,n_timesteps,n_train))
    # Simulate the LOpInf ROM for each episode
    for j in range(n_train):
        for k in range(n_timesteps):
            X_lopinf[:,k,j] = A_lopinf_dt@X_lopinf[:,k-1,j] + B_lopinf_dt@U_fom[:,k,j]
    # Compute the output matrices for the LOpInf ROM by projecting outputs onto states
    C_lopinf_ct = Y_fom[:,:,0].compute()@np.linalg.pinv(X_lopinf[:,:,0])
    # C_lopinf_ct = np.block([
    #     [Ehat_lopinf, np.zeros((m,n_ROM//2))]
    # ])
    D_lopinf_ct = np.zeros((m,l))

    C_lopinf_dt = C_lopinf_ct
    D_lopinf_dt = D_lopinf_ct
    # Save system matrices, projection mappings, and initial offset into file for later use
    np.savez(romDir+f"lopinfSystemMatrices_{n_ROM}dim_{n_train}train.npz", A_lopinf=A_lopinf_dt, B_lopinf=B_lopinf_dt, C_lopinf=C_lopinf_dt, D_lopinf=D_lopinf_dt, x0=x0, y0=y0, basis_lopinf=basis_lopinf)
    # save as .mat file
    scipy.io.savemat(romDir+f"lopinfSystemMatrices_{n_ROM}dim_{n_train}train.mat", mdict={'A_lopinf': A_lopinf_dt, 'B_lopinf': B_lopinf_dt, 'C_lopinf': C_lopinf_dt, 'D_lopinf': D_lopinf_dt, 'x0': x0, 'y0': y0, 'basis_lopinf': basis_lopinf})


In [7]:
Y_fom[:,:,0].compute().shape

(40, 2000)

In [8]:
X_lopinf[:,:,0].shape

(20, 2000)

In [9]:
np.linalg.pinv(X_lopinf[:,:,0]).shape

(2000, 20)